In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sco
import pandas as pd

In [2]:
vent = pd.read_csv("vent.csv", sep=";")
vent["time"] = pd.to_datetime(vent["time"], format="%Y-%m-%d %H:%M:%S")  # Make time to type time
vent = vent[vent["Id"] !=10] # Dropping line going up
vent = vent[vent["local time"] < 55]
# Needed to have one column regardless of which room the venting was in
def temperature_deduction(row):
    if row["room"] == "up":
        return row["temperature_bathoroom_upstairs"]
    elif row["room"] == "mid" or row["room"] == "middle":
        return row["temperature_bathroom_middle"]
    else:
        raise Exception("unkonwn floor: " + row["room"])
vent["temperature"] = vent.apply(lambda row: temperature_deduction(row), axis=1)

def config_code_to_size(row): # Converting the window code to the actual sizes
    code_mid = {1: 0.8*0.95, 2: 2*0.8*0.95, 3: 0.95*0.15 + 0.80*0.005, 4: 2*(0.95*0.15 + 0.80*0.005)}
    code_up = 0.6*0.9
    
    if row["room"] == "mid":
        return code_mid[row["Config"]]
    else:
        return code_up

vent["Config"] = vent.apply(lambda row: config_code_to_size(row), axis=1)

def config_code_to_size(row): # Converting the window code to the actual sizes
    if row["room"] == "mid":
        return 15*3
    else:
        return 10*3

vent["room"] = vent.apply(lambda row: config_code_to_size(row), axis=1)

def first_value(row):
    first_values_local =  vent[(row["Id"] == vent["Id"]) & (vent["local time"] == 0)]
    try:
        return first_values_local["absolute"].values[0], first_values_local["temperature"].values[0], first_values_local["absolute_outside"].values[0], first_values_local["temperature_outside"].values[0]
    except:
        return pd.NA, pd.NA, pd.NA, pd.NA

# Variables we only know from the first point in "production" so this is being simulated
first_values = vent.apply(
            lambda row: pd.Series(first_value(row), index=["humidity", "temperature", "humidity_outside",
                                                           "temperature_outside"]), axis=1)

relevant_vent = vent[["room", "Config", "local time"]]  # In the end this will be a minimum subset of data needed
# (column wise) so we do not drop too many lines removing NaNs

relevant_vent_hum = relevant_vent.copy()  # trainings data for the humidity model
relevant_vent_hum["humidity_goal"] = vent["absolute"]
relevant_vent_hum = pd.concat([first_values[["humidity_outside", "humidity"]], relevant_vent_hum], axis=1).dropna(axis=0, how="any")
relevant_vent_hum = relevant_vent_hum.apply(pd.to_numeric)

relevant_vent_temp = relevant_vent.copy()  # trainingsdata for the temperature model
relevant_vent_temp["temperature_goal"] = vent["temperature"]
relevant_vent_temp = pd.concat([first_values[["temperature", "temperature_outside"]], relevant_vent_temp], axis=1).dropna(axis=0, how="any")
relevant_vent_temp = relevant_vent_temp.apply(pd.to_numeric)

In [3]:
def h(h_0, h_out, F_größe, R_größe, t, k_1, k_2, k_3):  # The venting model
    #return (h_0 - h_out)*k_2 /(k_2 + t**((k_1*F_größe)/k_3*R_größe)) + h_out
    return (h_0 - h_out)*k_2 /(k_2 + t**((k_1*F_größe)/(k_3*R_größe))) + h_out

def T(T_0, T_out, F_größe, R_größe, t, k_0, k_1, k_2, k_3): # The temperature model
    return (T_0-(T_out*k_0))*(k_2)/(t**(k_1/(k_3*R_größe))+ k_2) + (T_out* k_0)

In [4]:
def get_F(T_0, T_out, F_größe, R_größe, t, L_in, func):
    def F(parameters):
        return sum((func(T_0, T_out, F_größe, R_größe, t, *parameters) - L_in)**2) # The sum of all errors through all measurements
    return F



# Functions generating the square error for given constants
error_sum_h = get_F(relevant_vent_hum["humidity"], relevant_vent_hum["humidity_outside"], 
                    relevant_vent_hum["Config"], 
                    relevant_vent_hum["room"], relevant_vent_hum["local time"], relevant_vent_hum["humidity_goal"], h)

error_sum_T = get_F(relevant_vent_temp["temperature"], relevant_vent_temp["temperature_outside"], 
                    relevant_vent_temp["Config"], 
                    relevant_vent_temp["room"], relevant_vent_temp["local time"], relevant_vent_temp["temperature_goal"], T)

In [5]:
start_values = [ 8.02021947, 13.4428695 ,  0.25453565]

res_h = sco.least_squares(error_sum_h, start_values)
res_h

 active_mask: array([0., 0., 0.])
        cost: 837.597689193579
         fun: array([40.92915072])
        grad: array([ 0.25531916,  6.64637486, -8.03739905])
         jac: array([[ 0.00623808,  0.16238731, -0.19637346]])
     message: 'The maximum number of function evaluations is exceeded.'
        nfev: 300
        njev: 289
  optimality: 8.037399050659632
      status: 0
     success: False
           x: array([ 8.02021433, 13.35220748,  0.25476616])

In [6]:
start_values = [1 , 6, 10, 0.1]

res_T = sco.least_squares(error_sum_T, start_values)
res_T

 active_mask: array([0., 0., 0., 0.])
        cost: 8076163.077114329
         fun: array([4018.99566487])
        grad: array([ 81885.64448193, -15510.01325825, -64242.31330554, 132269.00027173])
         jac: array([[ 20.3746536 ,  -3.85917641, -15.98466848,  32.91095868]])
     message: 'The maximum number of function evaluations is exceeded.'
        nfev: 400
        njev: 384
  optimality: 132269.00027173012
      status: 0
     success: False
           x: array([ 1.09504923, 13.04507119,  9.63370282,  1.5296808 ])